In [9]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [10]:
driver = webdriver.Chrome('chromedriver.exe')
url = 'https://www.imdb.com/title/tt9114286/reviews?ref_=tt_urv'
time.sleep(1)
driver.get(url)
time.sleep(1)

In [11]:
sel = Selector(text = driver.page_source)
review_counts = sel.css('.lister .header span::text').extract_first().replace(',','').split(' ')[0]
more_review_pages = int(int(review_counts)/25)

In [12]:
for i in tqdm(range(more_review_pages)):
    try:
        css_selector = 'load-more-trigger'
        driver.find_element(By.ID, css_selector).click()
    except:
        pass

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:00<00:00, 51.86it/s]


In [13]:
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')
first_review = reviews[0]
sel2 = Selector(text = first_review.get_attribute('innerHTML'))
rating = sel2.css('.rating-other-user-rating span::text').extract_first().strip()

In [14]:
rating_list = []
review_date_list = []
review_title_list = []
author_list = []
review_list = []
review_url_list = []
error_url_list = []
error_msg_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')

for d in tqdm(reviews):
    try:
        sel2 = Selector(text = d.get_attribute('innerHTML'))
        try:
            rating = sel2.css('.rating-other-user-rating span::text').extract_first()
        except:
            rating = np.NaN
        try:
            review = sel2.css('.text.show-more__control::text').extract_first()
        except:
            review = np.NaN
        try:
            review_date = sel2.css('.review-date::text').extract_first()
        except:
            review_date = np.NaN    
        try:
            author = sel2.css('.display-name-link a::text').extract_first()
        except:
            author = np.NaN    
        try:
            review_title = sel2.css('a.title::text').extract_first()
        except:
            review_title = np.NaN
        try:
            review_url = sel2.css('a.title::attr(href)').extract_first()
        except:
            review_url = np.NaN
        rating_list.append(rating)
        review_date_list.append(review_date)
        review_title_list.append(review_title)
        author_list.append(author)
        review_list.append(review)
        review_url_list.append(review_url)
    except Exception as e:
        error_url_list.append(url)
        error_msg_list.append(e)
review_df = pd.DataFrame({
    'Review_Date':review_date_list,
    'Author':author_list,
    'Rating':rating_list,
    'Review_Title':review_title_list,
    'Review':review_list,
    'Review_Url':review_url
    })


100%|████████████████████████████████████████████████████████████████████████████████| 275/275 [00:05<00:00, 49.02it/s]


In [15]:
review_df.head(100)

,Review_Date,Author,Rating,Review_Title,Review,Review_Url
0,14 November 2022,Drawmort,6,Not what I expected...\n,I am conflicted with this film and with Marvel.,/review/rw8670186/?ref_=tt_urv
1,27 November 2022,frankiesilver-91836,6,"A very bold but fairly underwhelming, slow, a...",I never thought the original Black Panther was...,/review/rw8670186/?ref_=tt_urv
2,26 November 2022,lord_orsum,6,"Too long, too muddled\n",Let's start with the good points - this film d...,/review/rw8670186/?ref_=tt_urv
3,18 November 2022,Kevin-42,None,Wakanda: It felt like forever!\n,"Sorry, but this was boring. Mind-numbing borin...",/review/rw8670186/?ref_=tt_urv
4,17 November 2022,slightlymad22,6,"Underwhelming, But OK\n",I just got out of Black Panther,/review/rw8670186/?ref_=tt_urv
...,...,...,...,...,...,...
95,17 December 2022,Special-K88,None,they tried but just couldn't replicate the ma...,Fans are taken back into the world of Wakanda ...,/review/rw8670186/?ref_=tt_urv
96,15 November 2022,eumesmozhoy,6,The wound of grief is still open in the Marve...,Grief is a wound that needs time to heal. In t...,/review/rw8670186/?ref_=tt_urv
97,16 November 2022,rchosen-193-5535,8,An movie that is almost was epic as a Avenger...,I loved the first movie of course and like man...,/review/rw8670186/?ref_=tt_urv
98,13 November 2022,guidocolato,6,Good ideas wasted\n,"This movie comes with some great ideas, most o...",/review/rw8670186/?ref_=tt_urv


In [16]:
review_df.to_csv('File.csv')